# Cilium

Cilium ist eine leistungsfähige und moderne Networking- und Security-Lösung für Kubernetes (K8s), die eBPF (Extended Berkeley Packet Filter) nutzt, um Netzwerk- und Sicherheitsrichtlinien effizient durchzusetzen. Es ersetzt herkömmliche Kubernetes-Netzwerk-Plugins (CNI) und bietet erweiterte Funktionen für Observability, Service Mesh und Security.

### Hauptmerkmale von Cilium:

1. **eBPF-basierte Netzwerksteuerung:**  
   - Direkte Manipulation von Netzwerkpaketen im Linux-Kernel für hohe Leistung und niedrige Latenz.
   - Effiziente und flexible Paketverarbeitung ohne iptables.

2. **Sicherheitsrichtlinien & Netzwerksegmentierung:**  
   - Anwendung von Identity-basierten (statt IP-basierten) Netzwerk-Sicherheitsrichtlinien.
   - Erlaubt Zero-Trust-Netzwerke und Microsegmentation in Kubernetes.

3. **Service Mesh ohne Sidecars:**  
   - Cilium kann als Alternative zu herkömmlichen Service-Mesh-Lösungen wie Istio dienen.
   - Vermeidet die Notwendigkeit, Sidecar-Proxies in jedem Pod zu betreiben.

4. **Observability & Netzwerk-Monitoring:**  
   - Bietet detaillierte Einblicke in den Kubernetes-Netzwerkverkehr mit eBPF-Tracing.
   - Ermöglicht Echtzeit-Analyse des Datenverkehrs und Fehlersuche.

5. **Multi-Cluster-Unterstützung:**  
   - Erlaubt Netzwerkverbindungen über verschiedene Kubernetes-Cluster hinweg.
   - Unterstützt Cluster Mesh für nahtlose Kommunikation.

6. **Hohe Performance & Skalierbarkeit:**  
   - Bessere Skalierbarkeit als herkömmliche Lösungen wie Calico oder Flannel.
   - Optimiert für moderne Cloud-native Anwendungen.

### Einsatzbereiche von Cilium:

- **Cloud-native Kubernetes-Netzwerke**: Sicheres und schnelles Networking für Microservices.
- **Security & Compliance**: Durchsetzung von Sicherheitsrichtlinien auf Netzwerkebene.
- **Service Mesh Alternative**: Vereinfachung von Service-Kommunikation ohne klassische Sidecars.
- **Observability & Debugging**: Überwachung und Analyse des Netzwerkverkehrs mit eBPF.

Cilium ist eine beliebte Wahl für Unternehmen, die Kubernetes-Cluster mit hoher Sicherheit, Skalierbarkeit und Performance betreiben möchten.

- - -

Zuerst erstellen wir einen Kubernetes Cluster mit `kind`

In [ ]:
%%bash
cat <<EOF >kind-config.yaml
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
  - role: control-plane
  - role: worker
  - role: worker
networking:
  disableDefaultCNI: true
EOF

kind create cluster --config=kind-config.yaml --name cilium

Dann Installieren wir das Cilium CLI

In [ ]:
%%bash
CILIUM_CLI_VERSION=$(curl -s https://raw.githubusercontent.com/cilium/cilium-cli/main/stable.txt)
CLI_ARCH=amd64
if [ "$(uname -m)" = "aarch64" ]; then CLI_ARCH=arm64; fi
curl -L --fail --remote-name-all https://github.com/cilium/cilium-cli/releases/download/${CILIUM_CLI_VERSION}/cilium-linux-${CLI_ARCH}.tar.gz{,.sha256sum}
sha256sum --check cilium-linux-${CLI_ARCH}.tar.gz.sha256sum
sudo tar xzvfC cilium-linux-${CLI_ARCH}.tar.gz /usr/local/bin
rm cilium-linux-${CLI_ARCH}.tar.gz{,.sha256sum}


Dann Cilium 

In [ ]:
%%bash
cilium install
cilium status --wait

Und das Cilium UI (hubble)

In [ ]:
%%bash
cilium hubble enable --ui

Port forward für UI
* Verbinden mit der VM
* Port forward nach "0.0.0.0" einrichten (weil kind Cluster in Docker Container läuft)
* URL im Browser öffnen

In [ ]:
%%bash
echo "ssh -i ~/.ssh/lerncloud ubuntu@$(cat ~/work/server-ip)"
echo "kubectl port-forward -n kube-system svc/hubble-ui --address 0.0.0.0 12000:80 &"
echo "Hubble UI: http://$(cat ~/work/server-ip):12000"

**Optional** Validieren wie die Installation

In [ ]:
%%bash
cilium connectivity test --request-timeout 30s --connect-timeout 10s

Und Erkunden die Ressourcen des Clusters

In [ ]:
%%bash
kubectl config get-contexts
kubectl get nodes
kubectl get daemonsets --all-namespaces
kubectl get deployments --all-namespaces 
kubectl get services  --all-namespaces 

### Weitere Hands-on

* [Hands-on: Overlay Netzwerk](02-overlay-network.ipynb)
* [Einfacher Service-Mesh](../C-Traffic%20Management/01-service-mesh.ipynb)
* [Service-Mesh - Big Ball of Mud](../C-Traffic%20Management/01-service-mesh-BBuM.ipynb)

**Hinweis**

Die angezeigten URLs zum Öffnen der Webshop-UIs und für den Lasttest funktionieren nicht, da der `kind`-Cluster in Docker-Containern läuft.  

Durch Port-Forwarding können wir die UIs dennoch zugänglich machen.  

Sobald wir etwas Traffic erzeugen (z. B. durch mehrmaliges Drücken von `Ctrl + F5` im Browser), werden die Verbindungen im Hubble UI sichtbar – ähnlich wie in Kiali oder Zipkin.  


In [ ]:
%%bash
echo "kubectl port-forward -n ms-rest service/webshop --address 0.0.0.0 12001:8080 &"
echo "kubectl port-forward -n ms-bbum service/webshop --address 0.0.0.0 12002:8080 &"

echo "Webshop UI (ms-rest): http://$(cat ~/work/server-ip):12001/webshop"
echo "Webshop UI (ms-bbum): http://$(cat ~/work/server-ip):12002/webshop"

- - -

### Aufräumen


In [ ]:
%%bash
kind delete cluster --name cilium
kubectl config use-context microk8s